In [2]:
import numpy as np
import pandas as pd

#Deep Learning
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Activation, Dropout, SimpleRNN, LSTM, GRU

#Data Setup
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

#Visualization
import matplotlib.pyplot as plt


C:\Users\HP\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#Initialization
#Setting up macros (based on question description)
MAX_FEATURES = 10000
MAX_LEN = 100
NUM_TOPICS = 24 
all_models=[]

In [4]:
#Importing the Data
df_train = pd.read_csv('data/congress_train.csv', encoding='ISO-8859-1').dropna()
df_test = pd.read_csv('data/congress_test.csv', encoding='ISO-8859-1').dropna()
df_valid = pd.read_csv('data/congress_val.csv', encoding='ISO-8859-1').dropna()

#Conversion to lists
txt_train = [str(word) for word in list(df_train['Title'])]
txt_test =  [str(word) for word in list(df_test['Title'])]
txt_valid = [str(word) for word in list(df_valid['Title'])] 

#Conversion to categorical
y_train = to_categorical(list(df_train['Major']))
y_test = to_categorical(list(df_test['Major']))
y_valid = to_categorical(list(df_valid['Major']))

In [5]:
#Tokenization
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(txt_train)
train_seq = tokenizer.texts_to_sequences(txt_train)
test_seq = tokenizer.texts_to_sequences(txt_test)
valid_seq = tokenizer.texts_to_sequences(txt_valid)

In [6]:
#Padding
X_train = pad_sequences(train_seq, maxlen=MAX_LEN)
X_test = pad_sequences(test_seq, maxlen=MAX_LEN)
X_valid = pad_sequences(valid_seq, maxlen=MAX_LEN)

In [7]:
#Estimate a basic feed-forward network
feedfwd = Sequential()
feedfwd.add(Embedding(10000, 25, input_length=100))
feedfwd.add(Flatten())
feedfwd.add(Dense(24, activation='softmax'))
feedfwd.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
result_feedfwd = feedfwd.fit(X_train, y_train, 
                             validation_data=(X_valid,y_valid), 
                             epochs=50, 
                             batch_size=512)
all_models.append(result_feedfwd)

Train on 278612 samples, validate on 69649 samples
Epoch 1/50
278612/278612 [==============================] - 8s 30us/step - loss: 1.8685 - acc: 0.4872 - val_loss: 1.0707 - val_acc: 0.7238
Epoch 2/50
278612/278612 [==============================] - 7s 26us/step - loss: 0.8441 - acc: 0.7789 - val_loss: 0.7402 - val_acc: 0.8032
Epoch 3/50
278612/278612 [==============================] - 9s 34us/step - loss: 0.6587 - acc: 0.8222 - val_loss: 0.6587 - val_acc: 0.8222
Epoch 4/50
278612/278612 [==============================] - 10s 35us/step - loss: 0.5856 - acc: 0.8410 - val_loss: 0.6211 - val_acc: 0.8329
Epoch 5/50
278612/278612 [==============================] - 12s 41us/step - loss: 0.5414 - acc: 0.8521 - val_loss: 0.6011 - val_acc: 0.8389
Epoch 6/50
278612/278612 [==============================] - 11s 39us/step - loss: 0.5096 - acc: 0.8602 - val_loss: 0.5895 - val_acc: 0.8436
Epoch 7/50
278612/278612 [==============================] - 9s 31us/step - loss: 0.4850 - acc: 0.8662 - val_loss

In [8]:
#Estimate a recurrent neural network (RNN) with a layer_simple_rnn
rnn = Sequential()
rnn.add(Embedding(10000, 20, input_length=100))
rnn.add(SimpleRNN(20))
rnn.add(Dense(24, activation='softmax'))
rnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
result_rnn = rnn.fit(X_train, y_train,
                     validation_data=(X_valid,y_valid),
                     epochs=50,
                     batch_size=512)
all_models.append(result_rnn)

Train on 278612 samples, validate on 69649 samples
Epoch 1/50
278612/278612 [==============================] - 29s 103us/step - loss: 2.5129 - acc: 0.2573 - val_loss: 2.2196 - val_acc: 0.3380
Epoch 2/50
278612/278612 [==============================] - 29s 104us/step - loss: 1.9792 - acc: 0.4245 - val_loss: 1.9906 - val_acc: 0.4290
Epoch 3/50
278612/278612 [==============================] - 29s 103us/step - loss: 1.7124 - acc: 0.5175 - val_loss: 1.6696 - val_acc: 0.5361
Epoch 4/50
278612/278612 [==============================] - 29s 104us/step - loss: 1.5501 - acc: 0.5709 - val_loss: 1.5678 - val_acc: 0.5676
Epoch 5/50
278612/278612 [==============================] - 30s 109us/step - loss: 1.4305 - acc: 0.6079 - val_loss: 1.4606 - val_acc: 0.6005
Epoch 6/50
278612/278612 [==============================] - 31s 112us/step - loss: 1.3313 - acc: 0.6373 - val_loss: 1.3893 - val_acc: 0.6244
Epoch 7/50
278612/278612 [==============================] - 30s 107us/step - loss: 1.2452 - acc: 0.6628

In [9]:
lstm = Sequential()
lstm.add(Embedding(10000, 20, input_length=100))
lstm.add(LSTM(20))
lstm.add(Dense(24, activation='softmax'))
lstm.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
result_lstm = lstm.fit(X_train, y_train,
                       validation_data=(X_valid,y_valid),
                       epochs=50,
                       batch_size=512)
all_models.append(result_lstm)


Train on 278612 samples, validate on 69649 samples
Epoch 1/50
278612/278612 [==============================] - 63s 228us/step - loss: 2.2371 - acc: 0.3366 - val_loss: 1.9794 - val_acc: 0.4285
Epoch 2/50
278612/278612 [==============================] - 58s 209us/step - loss: 1.3607 - acc: 0.6303 - val_loss: 1.1947 - val_acc: 0.6850
Epoch 3/50
278612/278612 [==============================] - 60s 214us/step - loss: 1.0221 - acc: 0.7397 - val_loss: 1.0216 - val_acc: 0.7390
Epoch 4/50
278612/278612 [==============================] - 58s 209us/step - loss: 0.8551 - acc: 0.7884 - val_loss: 0.8465 - val_acc: 0.7906
Epoch 5/50
278612/278612 [==============================] - 60s 214us/step - loss: 0.7529 - acc: 0.8128 - val_loss: 0.7781 - val_acc: 0.8038
Epoch 6/50
278612/278612 [==============================] - 67s 240us/step - loss: 0.6897 - acc: 0.8267 - val_loss: 0.7419 - val_acc: 0.8119
Epoch 7/50
278612/278612 [==============================] - 65s 233us/step - loss: 0.6455 - acc: 0.8355

In [11]:
#Estimate an RNN with a GRU layer
gru = Sequential()
gru.add(Embedding(10000, 20, input_length=100))
gru.add(GRU(20))
gru.add(Dense(24, activation='softmax'))
gru.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
result_gru = gru.fit(X_train, y_train, 
                     validation_data=(X_valid,y_valid), 
                     epochs=50, 
                     batch_size=512)
all_models.append(result_gru)

Train on 278612 samples, validate on 69649 samples
Epoch 1/50
278612/278612 [==============================] - 65s 233us/step - loss: 2.5482 - acc: 0.2022 - val_loss: 2.2541 - val_acc: 0.2825
Epoch 2/50
278612/278612 [==============================] - 68s 243us/step - loss: 1.9500 - acc: 0.4267 - val_loss: 1.6474 - val_acc: 0.5470
Epoch 3/50
278612/278612 [==============================] - 61s 219us/step - loss: 1.4306 - acc: 0.6269 - val_loss: 1.2593 - val_acc: 0.6863
Epoch 4/50
278612/278612 [==============================] - 72s 257us/step - loss: 1.1206 - acc: 0.7281 - val_loss: 1.1146 - val_acc: 0.7258
Epoch 5/50
278612/278612 [==============================] - 73s 263us/step - loss: 0.9441 - acc: 0.7694 - val_loss: 0.9395 - val_acc: 0.7672
Epoch 6/50
278612/278612 [==============================] - 63s 226us/step - loss: 0.8258 - acc: 0.7952 - val_loss: 0.8216 - val_acc: 0.7980
Epoch 7/50
278612/278612 [==============================] - 62s 224us/step - loss: 0.7458 - acc: 0.8148

In [ ]:
#Neural Network vs Hand-Coding

In [ ]:
rnn_dropout = Sequential()
rnn_dropout.add(Embedding(10000, 25, input_length=100))
rnn_dropout.add(SimpleRNN(25, dropout=0.2))
rnn_dropout.add(Dense(24, activation='softmax'))
rnn_dropout.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
result_rnn_dropout = rnn_dropout.fit(X_train, y_train,
                                     validation_data=(X_valid,y_valid),
                                     epochs=25,
                                     batch_size=512)
all_models.append(result_rnn_dropout)

In [ ]:
lstm_dropout = Sequential()
lstm_dropout.add(Embedding(10000, 25, input_length=100))
lstm_dropout.add(LSTM(25, dropout=0.2))
lstm_dropout.add(Dense(24, activation='softmax'))
lstm_dropout.compile(optimizer='rmsprop',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
result_lstm_dropout = lstm_dropout.fit(X_train, y_train,
                                       validation_data=(X_valid,y_valid),
                                       epochs=25,
                                       batch_size=512)
all_models.append(result_lstm_dropout)

In [ ]:
gru_dropout = Sequential()
gru_dropout.add(Embedding(10000, 25, input_length=100))
gru_dropout.add(GRU(25, dropout=0.2))
gru_dropout.add(Dense(24, activation='softmax'))
gru_dropout.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
result_gru_dropout = gru_dropout.fit(X_train, y_train,
                                     validation_data=(X_valid,y_valid),
                                     epochs=25,
                                     batch_size=512)
all_models.append(result_gru_dropout)

In [ ]:
rnn_2layer = Sequential()
rnn_2layer.add(Embedding(10000, 25, input_length=100))
rnn_2layer.add(SimpleRNN(25, return_sequences=True))
rnn_2layer.add(SimpleRNN(25))
rnn_2layer.add(Dense(24, activation='softmax'))
rnn_2layer.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
result_rnn_2layer = rnn_2layer.fit(X_train, y_train,
                                   validation_data=(X_valid,y_valid),
                                   epochs=25,
                                   batch_size=512)
all_models.append(result_rnn_2layer)

In [ ]:
rnn_3layer = Sequential()
rnn_3layer.add(Embedding(10000, 25, input_length=100))
rnn_3layer.add(SimpleRNN(25, return_sequences=True))
rnn_3layer.add(SimpleRNN(25, return_sequences=True))
rnn_3layer.add(SimpleRNN(25))
rnn_3layer.add(Dense(24, activation='softmax'))
rnn_3layer.compile(optimizer='rmsprop',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])
result_rnn_3layer = rnn_3layer.fit(X_train, y_train,
                                   validation_data=(X_valid,y_valid),
                                   epochs=25,
                                   batch_size=512)
all_models.append(result_rnn_3layer)

In [ ]:
def plot_acc(model):   
    plt.plot(model.history['val_acc'])

def plot_loss:
    plt.plot(model.history['val_loss'])
    

In [ ]:
#Combined Plot- Accuracies
for model in all_models:
    plot_acc(model)

plt.title('Accuracy on the Validation Set for All models Across Epochs')
plt.legend(['Basic Feed Forward', 'Basic RNN', 
            'Basic LSTM','Basic GRU',
            'RNN with Dropout', 
            'LSTM with Dropout','GRU with Dropout',
           'RNN with 2 layers', 'RNN with 3 layers'])
plt.show()

In [ ]:
#Combined Plot- Loss
for model in all_models:
    plot_loss(model)

plt.title('Loss on the Validation Set for All models Across Epochs')
plt.legend(['Basic Feed Forward', 'Basic RNN', 
            'Basic LSTM','Basic GRU',
            'RNN with Dropout', 
            'LSTM with Dropout','GRU with Dropout',
           'RNN with 2 layers', 'RNN with 3 layers'])
plt.show()


As per these plot, we can see that the highest performing one is the LSTM. 

In [13]:
#Now we use this model on the validation data
lstm.evaluate(X_test, y_test)

38693/38693 [==============================] - 9s 235us/step


[0.5719151524522886, 0.8529708215971779]

The accuracy is 